<a href="https://colab.research.google.com/github/cyberseihis/bnnprint/blob/main/notebooks/skqktnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install keras-tuner QKeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%pip install --no-deps scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
def get_csv_filenames():
    filenames = os.listdir()
    csv_filenames = [os.path.splitext(filename)[0] for filename in filenames if filename.endswith(".csv")]
    return csv_filenames

In [27]:
os.chdir('drive/MyDrive/dataset')

In [37]:
train_csv("pendigits")

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


164/164 [==============================] - 2s 4ms/step - loss: 2.8354
Epoch 2/100
164/164 [==============================] - 1s 5ms/step - loss: 1.8059
Epoch 3/100
164/164 [==============================] - 1s 5ms/step - loss: 1.5144
Epoch 4/100
164/164 [==============================] - 1s 5ms/step - loss: 1.2947
Epoch 5/100
164/164 [==============================] - 1s 5ms/step - loss: 1.3073
Epoch 6/100
164/164 [==============================] - 1s 4ms/step - loss: 1.1306
Epoch 7/100
164/164 [==============================] - 1s 3ms/step - loss: 1.0202
Epoch 8/100
164/164 [==============================] - 1s 3ms/step - loss: 0.9144
Epoch 9/100
164/164 [==============================] - 1s 3ms/step - loss: 0.8284
Epoch 10/100
164/164 [==============================] - 1s 3ms/step - loss: 0.7812
Epoch 11/100
164/164 [==============================] - 1s 3ms/step - loss: 0.7307
Epoch 12/100
164/164 [==============================] - 1s 3ms/step - loss: 0.7633
Epoch 13/100
164/164 [===

In [16]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import HyperParameters
import keras_tuner as kt
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import os

# data and metric imports
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

from qkeras import *
import qkeras.utils as qutils

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit

from scipy.stats import uniform

In [29]:
def get_model(meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    #model.add(QActivation("quantized_bits(4, keep_negative=0)"))
    model.add(QDense(units=40,kernel_quantizer="ternary", use_bias=False, input_shape=X_shape_[1:]))
    model.add(QActivation("binary"))
    model.add(QDense(units=n_classes_, kernel_quantizer="ternary(alpha=1)", use_bias=False))
    model.add(layers.Activation('softmax'))
    return model

In [32]:
def spit_csv(fname):
  df = pd.read_csv(f"{fname}.csv", header=None)
  X = df.iloc[:, :-1]
  y = df.iloc[:, -1]
  X = pd.DataFrame(X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))
  X = np.round(X*16).astype(int)
  X = np.minimum(15, X)
  X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
  (_, input_shape) = X.shape
  output_shape = y.nunique()
  y_train = to_categorical(y_train, num_classes=output_shape)
  y_test = to_categorical(y_test, num_classes=output_shape)
  return X_train, X_test, y_train, y_test

In [39]:
clf = KerasClassifier(
    get_model,
    epochs=100
)


params = {
    "loss": ["categorical_crossentropy"],
    #"optimizer": ["adam", "sgd"],
    "optimizer": ["adam"],
    "optimizer__learning_rate": [0.001, 0.002],
    "optimizer__beta_1": [0.9, 0.099],
    "optimizer__beta_2": [0.9, 0.099],
}

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
gs = GridSearchCV(clf, params, refit=True, cv=sss, scoring='accuracy',n_jobs=-1)

In [35]:
def train_csv(fname):
  X_train, X_test, y_train, y_test = spit_csv(fname)

  gs.fit(X_train, y_train)
  scors = f"{fname}:Test:{gs.score(X_test,y_test)},Train:{gs.best_score_}, {gs.best_params_}"
  with open('scors.txt', mode='a') as file:
    file.write(scors)
  model = gs.best_estimator_.model_
  qutils.model_save_quantized_weights(model,'weightnn'+fname+'.h5')
  model.save(fname+"tnn.h5")

In [ ]:
for fname in get_csv_filenames():
  train_csv(fname)

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


65/65 [==============================] - 1s 4ms/step - loss: 3.7651
Epoch 2/100
65/65 [==============================] - 0s 3ms/step - loss: 1.9446
Epoch 3/100
65/65 [==============================] - 0s 3ms/step - loss: 1.6857
Epoch 4/100
65/65 [==============================] - 0s 3ms/step - loss: 1.5118
Epoch 5/100
65/65 [==============================] - 0s 3ms/step - loss: 1.4486
Epoch 6/100
65/65 [==============================] - 0s 3ms/step - loss: 1.3498
Epoch 7/100
65/65 [==============================] - 0s 3ms/step - loss: 1.3084
Epoch 8/100
65/65 [==============================] - 0s 3ms/step - loss: 1.2951
Epoch 9/100
65/65 [==============================] - 0s 3ms/step - loss: 1.3423
Epoch 10/100
65/65 [==============================] - 0s 3ms/step - loss: 1.2974
Epoch 11/100
65/65 [==============================] - 0s 3ms/step - loss: 1.3133
Epoch 12/100
65/65 [==============================] - 0s 3ms/step - loss: 1.3416
Epoch 13/100
65/65 [=============================

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


35/35 [==============================] - 1s 3ms/step - loss: 4.7704
Epoch 2/100
35/35 [==============================] - 0s 3ms/step - loss: 2.6135
Epoch 3/100
35/35 [==============================] - 0s 3ms/step - loss: 1.9575
Epoch 4/100
35/35 [==============================] - 0s 3ms/step - loss: 1.7661
Epoch 5/100
35/35 [==============================] - 0s 3ms/step - loss: 1.6634
Epoch 6/100
35/35 [==============================] - 0s 3ms/step - loss: 1.6482
Epoch 7/100
35/35 [==============================] - 0s 3ms/step - loss: 1.6271
Epoch 8/100
35/35 [==============================] - 0s 4ms/step - loss: 1.6682
Epoch 9/100
35/35 [==============================] - 0s 3ms/step - loss: 1.6964
Epoch 10/100
35/35 [==============================] - 0s 4ms/step - loss: 1.5854
Epoch 11/100
35/35 [==============================] - 0s 3ms/step - loss: 1.6115
Epoch 12/100
35/35 [==============================] - 0s 3ms/step - loss: 1.5518
Epoch 13/100
35/35 [=============================

...

108/108 [==============================] - 1s 3ms/step - loss: 2.2323
Epoch 2/100
108/108 [==============================] - 0s 3ms/step - loss: 1.8583
Epoch 3/100
108/108 [==============================] - 0s 3ms/step - loss: 1.7704
Epoch 4/100
108/108 [==============================] - 0s 3ms/step - loss: 1.5884
Epoch 5/100
108/108 [==============================] - 0s 3ms/step - loss: 1.6514
Epoch 6/100
108/108 [==============================] - 0s 4ms/step - loss: 1.6453
Epoch 7/100
108/108 [==============================] - 0s 4ms/step - loss: 1.6686
Epoch 8/100
108/108 [==============================] - 0s 4ms/step - loss: 1.6463
Epoch 9/100
108/108 [==============================] - 1s 5ms/step - loss: 1.6309
Epoch 10/100
108/108 [==============================] - 1s 5ms/step - loss: 1.6302
Epoch 11/100
108/108 [==============================] - 1s 5ms/step - loss: 1.6864
Epoch 12/100
108/108 [==============================] - 1s 5ms/step - loss: 1.6589
Epoch 13/100
108/108 [===

/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


305/305 [==============================] - 2s 3ms/step - loss: 1.7590
Epoch 2/100
305/305 [==============================] - 1s 4ms/step - loss: 1.3954
Epoch 3/100
305/305 [==============================] - 1s 4ms/step - loss: 1.3671
Epoch 4/100
305/305 [==============================] - 1s 4ms/step - loss: 1.3597
Epoch 5/100
305/305 [==============================] - 1s 3ms/step - loss: 1.3546
Epoch 6/100
305/305 [==============================] - 1s 4ms/step - loss: 1.2598
Epoch 7/100
305/305 [==============================] - 1s 4ms/step - loss: 1.2786
Epoch 8/100
305/305 [==============================] - 1s 3ms/step - loss: 1.2306
Epoch 9/100
305/305 [==============================] - 1s 4ms/step - loss: 1.1848
Epoch 10/100
305/305 [==============================] - 2s 5ms/step - loss: 1.2375
Epoch 11/100
305/305 [==============================] - 2s 5ms/step - loss: 1.1854
Epoch 12/100
305/305 [==============================] - 1s 4ms/step - loss: 1.1751
Epoch 13/100
305/305 [===

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
pd.DataFrame(gs.cv_results_)[['param_optimizer__learning_rate','param_optimizer__beta_1','param_optimizer__beta_2','mean_test_score']]

,param_optimizer__learning_rate,param_optimizer__beta_1,param_optimizer__beta_2,mean_test_score
0,0.001,0.9,0.9,0.841169
1,0.002,0.9,0.9,0.814168
2,0.001,0.9,0.099,0.627065
3,0.002,0.9,0.099,0.147395


In [13]:
bpr={'loss': 'categorical_crossentropy', 'optimizer': 'adam', 'optimizer__beta_1': 0.9, 'optimizer__beta_2': 0.9, 'optimizer__learning_rate': 0.002}
clfx = KerasClassifier(
    get_model,
    epochs=100,
    **bpr
)
clfx.fit(X_train,y_train)

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


164/164 [==============================] - 1s 3ms/step - loss: 2.8603
Epoch 2/100
164/164 [==============================] - 1s 3ms/step - loss: 1.0766
Epoch 3/100
164/164 [==============================] - 1s 4ms/step - loss: 0.8790
Epoch 4/100
164/164 [==============================] - 1s 3ms/step - loss: 0.7852
Epoch 5/100
164/164 [==============================] - 1s 3ms/step - loss: 0.7272
Epoch 6/100
164/164 [==============================] - 1s 3ms/step - loss: 0.6506
Epoch 7/100
164/164 [==============================] - 1s 3ms/step - loss: 0.6188
Epoch 8/100
164/164 [==============================] - 1s 3ms/step - loss: 0.6365
Epoch 9/100
164/164 [==============================] - 1s 3ms/step - loss: 0.6577
Epoch 10/100
164/164 [==============================] - 1s 3ms/step - loss: 0.6760
Epoch 11/100
164/164 [==============================] - 1s 3ms/step - loss: 0.6564
Epoch 12/100
164/164 [==============================] - 1s 4ms/step - loss: 0.6575
Epoch 13/100
164/164 [===

KerasClassifier(
	model=<function get_model at 0x7f222024de50>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=categorical_crossentropy
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	optimizer__beta_1=0.9
	optimizer__beta_2=0.9
	optimizer__learning_rate=0.002
	class_weight=None
)

In [14]:
clfx.score(X_test,y_test)

71/71 [==============================] - 1s 3ms/step


0.8875055580257892

In [12]:
pd.DataFrame(gs.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'loss': 'categorical_crossentropy', 'optimize...",0.852287
1,"{'loss': 'categorical_crossentropy', 'optimize...",0.726811
2,"{'loss': 'categorical_crossentropy', 'optimize...",0.879606
3,"{'loss': 'categorical_crossentropy', 'optimize...",0.824651
4,"{'loss': 'categorical_crossentropy', 'optimize...",0.741741
5,"{'loss': 'categorical_crossentropy', 'optimize...",0.858323
6,"{'loss': 'categorical_crossentropy', 'optimize...",0.817662
7,"{'loss': 'categorical_crossentropy', 'optimize...",0.654384
8,"{'loss': 'categorical_crossentropy', 'optimize...",0.807814
9,"{'loss': 'categorical_crossentropy', 'optimize...",0.827510


AttributeError: ignored

In [ ]:
for fname in get_csv_filenames():
  train_csv(fname)

less_pendigitssummary


1/1 [==============================] - 1s 681ms/step
... quantizing model
71/71 [==============================] - 1s 3ms/step - loss: 0.5281 - accuracy: 0.8395
less_cardiosummary


1/1 [==============================] - 1s 683ms/step
... quantizing model
20/20 [==============================] - 0s 4ms/step - loss: 0.4019 - accuracy: 0.8824
less_Harsummary


1/1 [==============================] - 1s 682ms/step
... quantizing model
28/28 [==============================] - 0s 4ms/step - loss: 1.1668 - accuracy: 0.5333
less_winequality_redsummary


1/1 [==============================] - 1s 663ms/step
... quantizing model
15/15 [==============================] - 0s 3ms/step - loss: 1.2332 - accuracy: 0.5708
less_gasIdsummary


1/1 [==============================] - 1s 669ms/step
... quantizing model
131/131 [==============================] - 1s 3ms/step - loss: 0.5700 - accuracy: 0.8212
less_winequality_whitesummary


1/1 [==============================] - 1s 664ms/step
... quantizing model
46/46 [==============================] - 0s 3ms/step - loss: 1.6193 - accuracy: 0.5048


In [ ]:
df = pd.read_csv('winequality_white.csv', header=None)
Xx = df.iloc[:, :-1]
y = df.iloc[:, -1]
X = pd.DataFrame(Xx.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))

In [ ]:
%pip install h5py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.models import load_model

# Load the model from the HDF5 file
model = load_model('winequality_whitetnn.h5', custom_objects={'QActivation': QActivation,'QDense':QDense})

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
#x_hack = tf.random.normal(shape=(1,16),dtype=tf.float32)
#x_hack = tf.abs(x_hack)
#output=model.predict(x_hack)
#inpux = np.expand_dims(X.iloc[0], axis=0)
inpux = X.iloc[0:1]
lo=get_layer_outputs(model,inpux)
yoyo = lo[2]
gol = lo[4]
lo

[array([[0.3125, 0.1875, 0.1875, 0.3125, 0.125 , 0.125 , 0.375 , 0.25  ,
         0.25  , 0.25  , 0.125 ]], dtype=float32),
 array([[-0.15625 ,  1.625   , -0.03125 , -0.25    ,  0.09375 , -0.1875  ,
          0.59375 , -0.25    , -0.09375 ,  0.      ,  0.28125 ,  0.15625 ,
          0.1875  ,  0.15625 ,  0.03125 , -0.21875 ,  1.9375  , -0.1875  ,
          0.234375, -0.21875 ,  0.      ,  0.15625 ,  0.125   ,  1.625   ,
          0.0625  , -0.0625  , -0.3125  ,  0.4375  , -0.0625  , -0.4375  ,
          0.4375  ,  0.      ,  0.53125 ,  0.28125 , -0.09375 ,  0.21875 ,
         -0.25    , -0.125   ,  0.125   , -0.078125]], dtype=float32),
 array([[-1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
          1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
         -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,
         -1.]], dtype=float32),
 array([[-6., -3.,  0.,  2., -3., -4., -5.]], dtype=float32),
 array([[2.9106444e-04, 5.8461856e-03, 

In [ ]:
xrisos = np.argmax(gol,axis=1)
qinp = lo[0]

In [ ]:
import h5py

filename = "weightnnwinequality_white.h5"

h5 = h5py.File(filename,'r')

h5.visit(print)
#print(h5['q_dense/q_dense/kernel:0'][:,:])
#print(h5['q_batch_normalization/q_batch_normalization/beta:0'][:])
sw0 = h5['q_dense/q_dense/kernel:0'][:,:]
sw1 = h5['q_dense_1/q_dense_1/kernel:0'][:,:]

h5.close()

activation
q_activation
q_activation_1
q_dense
q_dense/q_dense
q_dense/q_dense/kernel:0
q_dense_1
q_dense_1/q_dense_1
q_dense_1/q_dense_1/kernel:0
top_level_model_weights


In [ ]:
matz=np.sign(sw0)
abz=np.abs(matz)
sinz=np.maximum(matz,0)
binst(abz.flatten())[::-1]

'10011001110000000100111010000101000100011001101110011010111111111100011111111110011001101001111010110011101101001011101001001110011000111011011011110110101101001111101101010110110110010101001010110101001110011011110011011011100001110110110101110111111100111011101110010000001111011111011101101011111011011100011111011001101000001101010101100111000001000001000111101010100110101001010000010110000111000110100101011110101011111110110100000011'

In [ ]:
cnz=np.count_nonzero(sw0, axis=0)
''.join([f"0{i}" for i in cnz])

'06070806070407070707040806050804090607070707080808050305010906050503060906060307'

In [ ]:
hay=binst(yoyo[0])
jay = '0100101001111110101011111001001111010010'[::-1]
jay

'0100101111001001111101010111111001010010'

In [ ]:
def dbytes(mat):
  kay=mat.flatten().astype(np.uint8).tobytes()
  return binascii.hexlify(kay).decode()

In [ ]:
from scipy.sparse import csr_matrix
spa=csr_matrix(sw1.T)
#spa.data,spa.indices,spa.indptr
svals=binst(np.maximum(spa.data,0))
scols=dbytes(spa.indices[::-1])
srows=dbytes(spa.indptr[::-1])
svals,scols,srows

('0100000010110010011000100000110000101111111100111011000110111010101011011001010',
 '2625232221201e1b1817141211100d0c0605040201211f17100d0b211c1615100c0825211f1817100f0125242119181412092625221f1e1c1817161413100b0a07261c171412100f0b0a0907060401',
 '4f3a342d251d0e00')

In [ ]:
lo[3]

array([[-6., -3.,  0.,  2., -3., -4., -5.]], dtype=float32)

In [ ]:
sw1.T

array([[ 0., -1.,  0.,  0.,  1.,  0., -1.,  1.,  0., -1., -1.,  1.,  0.,
         0.,  0.,  1., -1.,  0.,  1.,  0.,  1.,  0.,  0., -1.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  1.,  0.,
         0.,  0.,  0., -1.,  0.,  0.,  1.,  1.,  0.,  1., -1.,  1.,  0.,
         0.,  0.,  1.,  0., -1., -1.,  0.,  0., -1.,  0.,  0.,  1.,  1.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1., -1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  1.,  0.,
         0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0., 

In [ ]:
nn=np.count_nonzero(sw1, axis=0)
oh=(lo[3]+nn)/2
lo[3],oh

(array([[-6., -3.,  0.,  2., -3., -4., -5.]], dtype=float32),
 array([[4., 6., 4., 5., 2., 1., 8.]]))

In [ ]:
np.max(np.abs(sw0.T),axis=1)

array([0.25, 1.  , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.25, 0.5 ,
       0.25, 0.5 , 0.5 , 0.5 , 0.5 , 1.  , 0.5 , 0.25, 0.5 , 0.5 , 0.5 ,
       0.5 , 1.  , 0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.25, 0.5 , 0.5 , 0.5 , 0.25], dtype=float32)

In [ ]:
def mysign(a):
  signs = np.sign(a)

  # Replace the sign of the zeroes with 1 (positive) or -1 (negative)
  signs_no_zero = np.where(signs == 0, 1, signs)
  return signs_no_zero

In [ ]:
def grup(x):
  x1 = x @ sw0
  x2 = mysign(x1)
  x3 = x2 @ sw1
  return (x1,x2,x3)
def drut(x):
  x1 = x @ np.sign(sw0)
  x2 = mysign(x1)
  x3 = x2 @ sw1
  am = np.argmax(x3,axis=1)
  return am

In [ ]:
xrisos[0:20]

array([3, 3, 2, 3, 3])

In [ ]:
drut(qinp)[0:20]

array([3, 3, 2, 3, 3])

In [ ]:
np.where(np.not_equal(yoyo, grup(qinp)[1]))

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
import binascii
def binst(yy):
  yozo = np.maximum(yy,0)
  asbin=yozo.astype(np.uint8).tobytes()
  bs=binascii.hexlify(asbin).decode()[1::2]
  return bs[::-1]

[print(binst(yy)) for yy in yoyo]

0100101111001001111101010111111001010010


[None]

In [ ]:
y[0:10]

0    3
1    3
2    3
3    3
4    3
5    3
6    3
7    3
8    3
9    3
Name: 11, dtype: int64

In [ ]:
glo = lo[2][175]
gor = grup(qinp[175])[1]
glo, gor, np.where(np.not_equal(glo,gor))

(array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,
        -1.], dtype=float32),
 array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  0.,  0., -1.,  1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  0., -1.,  1., -1., -1., -1., -1.,
        -1.], dtype=float32),
 (array([ 9, 10, 32]),))

In [ ]:
#dut = yoyo @ (np.sign(swax))
#dd , gg = np.argmax(dut,axis=1),np.argmax(gol,axis=1)
hm=np.where(np.not_equal(drut(qinp),y[0:199]))
ex=np.where(np.not_equal(xrisos,y[0:199]))
#(dd[hm],gg[hm],hm)
hm,ex,np.where(np.not_equal(hm,ex))

((array([ 13,  27,  29,  49,  74,  82,  86,  93, 114, 148, 152, 154, 160,
         161, 162, 166, 180]),),
 (array([ 13,  27,  29,  49,  74,  82,  86,  93, 114, 148, 152, 154, 160,
         161, 162, 166, 180]),),
 (array([], dtype=int64), array([], dtype=int64)))

In [ ]:
h5.keys()

ValueError: ignored

In [ ]:
h5['q_dense_1/q_dense_1/kernel:0'][:,:]

ValueError: ignored

In [ ]:
list(QBatchNormalization().quantizers)

In [ ]:
import keras.backend as K

def get_layer_outputs(model, x):
    """
    Returns the output of every layer in a Keras model evaluated at the input x.
    
    Args:
    model: A Keras model object.
    x: The input to the model as a Numpy array or a list of Numpy arrays.
    
    Returns:
    A list of Numpy arrays, where the i-th element is the output of the i-th layer
    in the model evaluated at the input x.
    """
    # Create a function that computes the output of every layer given the input
    layer_outputs = [layer.output for layer in model.layers]
    layer_eval_fn = K.function(inputs=[model.input], outputs=layer_outputs)
    
    # Evaluate the function at the input
    output_values = layer_eval_fn([x])
    
    return output_values


In [ ]:
from keras.models import load_model
def get_results(fname):
    df = pd.read_csv(fname+".csv", header=None)
    X = df.iloc[:, :-1]
    
    X = pd.DataFrame(X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))
    
    model = load_model(fname+"tnn.h5", custom_objects={'QActivation': QActivation,'QDense':QDense})
   
    output=model.predict(X)
    clases = np.argmax(output, axis=1)
    return clases

In [ ]:
get_results("pendigits")[0:14]

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


235/235 [==============================] - 4s 2ms/step


array([8, 2, 1, 4, 1, 6, 4, 0, 5, 0, 9, 8, 5, 5])

In [ ]:
def write_strings_to_file(strings, file_path):
    with open(file_path, 'w') as file:
        for string in strings:
            file.write(str(string) + '\n')

In [ ]:
for fname in get_csv_filenames():
  gres = get_results(fname).tolist()
  write_strings_to_file(gres,'results_'+fname+'.results')

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


235/235 [==============================] - 1s 3ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


67/67 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


...

50/50 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


435/435 [==============================] - 1s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


93/93 [==============================] - 0s 2ms/step


In [ ]:
inpux = X.iloc[0:1]
#inpux = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
lo=get_layer_outputs(model,inpux)
yoyo = lo[2]
gol = lo[4]
lo